In [1]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

# Import libraries for feature selection
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
#from mlxtend.feature_selection import SequentialFeatureSelector as SFS

import warnings
warnings.filterwarnings("ignore")


In [2]:
#Read training data file
trainfile = r'C:/Users/farha/Downloads/Santander Customer Satisfaction - TRAIN.csv'
train_data = pd.read_csv(trainfile)
testfile = r'C:/Users/farha/Downloads/Santander Customer Satisfaction - TEST-Without TARGET.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 
print()
print(test_data.shape)
print(test_data.head()) 


(76020, 371)
   ID  var3  var15  imp_ent_var16_ult1  imp_op_var39_comer_ult1  \
0   1     2     23                 0.0                      0.0   
1   3     2     34                 0.0                      0.0   
2   4     2     23                 0.0                      0.0   
3   8     2     37                 0.0                    195.0   
4  10     2     39                 0.0                      0.0   

   imp_op_var39_comer_ult3  imp_op_var40_comer_ult1  imp_op_var40_comer_ult3  \
0                      0.0                      0.0                      0.0   
1                      0.0                      0.0                      0.0   
2                      0.0                      0.0                      0.0   
3                    195.0                      0.0                      0.0   
4                      0.0                      0.0                      0.0   

   imp_op_var40_efect_ult1  imp_op_var40_efect_ult3  ...  \
0                      0.0                 

In [3]:
Traincols = list(train_data.columns.values)
Testcols = list(test_data.columns.values)

In [4]:
Xtrain = train_data[Traincols[0:len(Traincols)-1]].copy()
Ytrain = train_data[['TARGET']]
print(train_data.value_counts)
Xtest = test_data

<bound method DataFrame.value_counts of            ID  var3  var15  imp_ent_var16_ult1  imp_op_var39_comer_ult1  \
0           1     2     23                 0.0                      0.0   
1           3     2     34                 0.0                      0.0   
2           4     2     23                 0.0                      0.0   
3           8     2     37                 0.0                    195.0   
4          10     2     39                 0.0                      0.0   
...       ...   ...    ...                 ...                      ...   
76015  151829     2     48                 0.0                      0.0   
76016  151830     2     39                 0.0                      0.0   
76017  151835     2     23                 0.0                      0.0   
76018  151836     2     25                 0.0                      0.0   
76019  151838     2     46                 0.0                      0.0   

       imp_op_var39_comer_ult3  imp_op_var40_comer_ult1  \


In [5]:
print(Ytrain.value_counts())

TARGET
0         73012
1          3008
dtype: int64


In [6]:
from imblearn import under_sampling
from imblearn.under_sampling import  RandomUnderSampler

under_sampler = RandomUnderSampler(sampling_strategy=1)
X_res, y_res = under_sampler.fit_resample(Xtrain, Ytrain)

In [7]:
print(y_res.value_counts())

TARGET
0         3008
1         3008
dtype: int64


In [8]:
#GENETIC ALGORITHM FOR FEATURE SELECTION ===================================================================================
#===========================================================================================================================
#https://pypi.org/project/sklearn-genetic/

from genetic_selection import GeneticSelectionCV



clf = RandomForestClassifier()
model = GeneticSelectionCV(
    clf, cv=2, verbose=0,
    scoring="accuracy", max_features=30,
    n_population=100, crossover_proba=0.5,
    mutation_proba=0.2, n_generations=100,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model = model.fit(X_res, y_res)



In [9]:
#Do feature selection using the genetic algo
#Get column names
#print(model.get_support())

#Get column names
cols = model.get_support(indices=True) #get column indices
print("\n cols = ", cols, "\n")

features_df_new_ga = X_res.iloc[:,cols] #Get cols of X-Train with col names
print(features_df_new_ga.shape)
print ("\n features_df_new_ga= ", features_df_new_ga)
#print(features_df_new.head())

#save file



 cols =  [  1   2   6  14  27  36  50  73  92 105 111 158 160 162 163 165 167 169
 175 183 241 246 298 307 326 336 363 364 367] 

(6016, 29)

 features_df_new_ga=        var3  var15  imp_op_var40_comer_ult1  imp_op_var41_efect_ult3  ind_var6  \
0        2     38                      0.0                      0.0         0   
1        2     40                      0.0                      0.0         0   
2        2     23                      0.0                      0.0         0   
3        4     28                      0.0                    210.0         0   
4        2     23                      0.0                      0.0         0   
...    ...    ...                      ...                      ...       ...   
6011     2     30                      0.0                      0.0         0   
6012     2     27                      0.0                    540.0         0   
6013     2     26                      0.0                      0.0         0   
6014     2     75        

In [10]:
new_X_train_SelectedFeatures_GA= pd.DataFrame(features_df_new_ga)
#print(new_X_train_SelectedFeatures_GA)
new_X_train_SelectedFeatures_GA

,var3,var15,imp_op_var40_comer_ult1,imp_op_var41_efect_ult3,ind_var6,ind_var13_largo,ind_var24,ind_var34,num_var6_0,num_var13,...,saldo_var30,imp_var43_emit_ult1,imp_trasp_var17_out_ult1,num_op_var41_comer_ult1,num_reemb_var17_ult1,num_var45_ult1,saldo_medio_var8_ult3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_ult1
0,2,38,0.0,0.0,0,0,0,0,0,0,...,300003.00,0.0,0.0,0,0,21,0.0,0.0,0.0,0.0
1,2,40,0.0,0.0,0,0,0,0,0,0,...,150.00,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
2,2,23,0.0,0.0,0,0,0,0,0,0,...,0.00,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
3,4,28,0.0,210.0,0,0,0,0,0,3,...,6222.33,2745.0,0.0,12,0,9,0.0,0.0,0.0,0.0
4,2,23,0.0,0.0,0,0,0,0,0,0,...,3.00,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6011,2,30,0.0,0.0,0,0,0,0,0,0,...,0.00,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
6012,2,27,0.0,540.0,0,0,0,0,0,0,...,37.80,0.0,0.0,21,0,9,0.0,0.0,0.0,0.0
6013,2,26,0.0,0.0,0,0,0,0,0,0,...,0.00,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
6014,2,75,0.0,0.0,0,0,0,0,0,0,...,150.00,0.0,0.0,0,0,9,0.0,0.0,0.0,0.0


In [11]:
#GENETIC ALGORITHM FOR FEATURE SELECTION ===================================================================================
#===========================================================================================================================
#https://pypi.org/project/sklearn-genetic/

from genetic_selection import GeneticSelectionCV



clf = RandomForestClassifier()
model_50 = GeneticSelectionCV(
    clf, cv=2, verbose=0,
    scoring="accuracy", max_features=55,
    n_population=100, crossover_proba=0.5,
    mutation_proba=0.2, n_generations=100,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model_50 = model_50.fit(X_res, y_res)



In [12]:
#Do feature selection using the genetic algo
#Get column names
#print(model.get_support())

#Get column names
cols = model_50.get_support(indices=True) #get column indices
print("\n cols = ", cols, "\n")

features_df_new_ga_50 = X_res.iloc[:,cols] #Get cols of X-Train with col names
print(features_df_new_ga_50.shape)
print ("\n features_df_new_ga= ", features_df_new_ga_50)
#print(features_df_new.head())

#save file



 cols =  [  1   2   8  14  36  58  64  72  76  83 117 124 137 144 147 157 158 162
 171 183 184 191 200 211 221 225 229 238 253 258 259 270 275 278 280 290
 295 302 308 309 329 333 352 366] 

(6016, 44)

 features_df_new_ga=        var3  var15  imp_op_var40_efect_ult1  imp_op_var41_efect_ult3  \
0        2     38                      0.0                      0.0   
1        2     40                      0.0                      0.0   
2        2     23                      0.0                      0.0   
3        4     28                      0.0                    210.0   
4        2     23                      0.0                      0.0   
...    ...    ...                      ...                      ...   
6011     2     30                      0.0                      0.0   
6012     2     27                      0.0                    540.0   
6013     2     26                      0.0                      0.0   
6014     2     75                      0.0                      

In [8]:
#GENETIC ALGORITHM FOR FEATURE SELECTION ===================================================================================
#===========================================================================================================================
#https://pypi.org/project/sklearn-genetic/

from genetic_selection import GeneticSelectionCV



clf = GradientBoostingClassifier()
model = GeneticSelectionCV(
    clf, cv=2, verbose=0,
    scoring="accuracy", max_features=30,
    n_population=100, crossover_proba=0.5,
    mutation_proba=0.2, n_generations=100,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model = model.fit(X_res, y_res)



In [9]:
#Do feature selection using the genetic algo
#Get column names
#print(model.get_support())

#Get column names
cols = model.get_support(indices=True) #get column indices
print("\n cols = ", cols, "\n")

features_df_new_ga = X_res.iloc[:,cols] #Get cols of X-Train with col names
print(features_df_new_ga.shape)
print ("\n features_df_new_ga= ", features_df_new_ga)
#print(features_df_new.head())

#save file



 cols =  [  2  13  14  53  74  80  86  96  99 119 154 183 194 201 208 228 230 278
 293 319 324 327 329 361 369] 

(6016, 25)

 features_df_new_ga=        var15  imp_op_var41_efect_ult1  imp_op_var41_efect_ult3  ind_var26_cte  \
0        29                      0.0                      0.0              0   
1        38                      0.0                      0.0              1   
2        28                      0.0                      0.0              0   
3        61                      0.0                      0.0              1   
4        23                      0.0                      0.0              0   
...     ...                      ...                      ...            ...   
6011     30                      0.0                      0.0              0   
6012     27                    540.0                    540.0              0   
6013     26                      0.0                      0.0              0   
6014     75                      0.0               

In [10]:
#GENETIC ALGORITHM FOR FEATURE SELECTION ===================================================================================
#===========================================================================================================================
#https://pypi.org/project/sklearn-genetic/

from genetic_selection import GeneticSelectionCV



clf = GradientBoostingClassifier()
model_50 = GeneticSelectionCV(
    clf, cv=2, verbose=0,
    scoring="accuracy", max_features=55,
    n_population=100, crossover_proba=0.5,
    mutation_proba=0.2, n_generations=100,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model_50 = model_50.fit(X_res, y_res)



In [11]:
#Do feature selection using the genetic algo
#Get column names
#print(model.get_support())

#Get column names
cols = model_50.get_support(indices=True) #get column indices
print("\n cols = ", cols, "\n")

features_df_new_ga_50 = X_res.iloc[:,cols] #Get cols of X-Train with col names
print(features_df_new_ga_50.shape)
print ("\n features_df_new_ga= ", features_df_new_ga_50)
#print(features_df_new.head())

#save file



 cols =  [  2  16  17  29  33  34  53  59  60  62  63  72  83  90  94 101 105 113
 114 148 158 163 178 183 188 192 194 202 231 237 249 251 257 273 276 278
 287 292 298 299 305 310 324 325 330 332 349 353 365 367 369] 

(6016, 51)

 features_df_new_ga=        var15  imp_op_var39_efect_ult1  imp_op_var39_efect_ult3  ind_var8  \
0        29                      0.0                      0.0         0   
1        38                      0.0                      0.0         1   
2        28                      0.0                      0.0         0   
3        61                      0.0                      0.0         1   
4        23                      0.0                      0.0         0   
...     ...                      ...                      ...       ...   
6011     30                      0.0                      0.0         0   
6012     27                    540.0                    540.0         0   
6013     26                      0.0                      0.0         0

In [13]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(param)

{'n_estimators': [200, 233, 266, 300, 333, 366, 400, 433, 466, 500], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [14]:
from tpot import TPOTClassifier


tpot_classifier = TPOTClassifier(generations= 5, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestClassifier': param}, 
                                 cv = 4, scoring = 'accuracy')
tpot_classifier.fit(features_df_new_ga,y_res)

Optimization Progress:   0%|          | 0/84 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7480053191489362

Generation 2 - Current best internal CV score: 0.7480053191489362

Generation 3 - Current best internal CV score: 0.7485039893617021

Generation 4 - Current best internal CV score: 0.749501329787234

Generation 5 - Current best internal CV score: 0.749501329787234

Best pipeline: RandomForestClassifier(CombineDFs(input_matrix, input_matrix), criterion=gini, max_depth=450, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=300)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'criterion': ['entropy',
                                                                                      'gini'],
                                                                        'max_depth': [10,
                                                                                      120,
                                                                                      230,
                                                                                      340,
                                                                                      450,
                                                                                      560,
                                                                                      670,
                                                                                      780,
                                                                                      890,
                                                                                      1000],
                                                                        'max_features': ['auto',
                                                                                         'sqrt',
                                                                                         'log2'],
                                                                        'min_samples_leaf': [1,
                                                                                             2,
                                                                                             4,
                                                                                             6,
                                                                                             8],
                                                                        'min_samples_split': [2,
                                                                                              5,
                                                                                              10,
                                                                                              14],
                                                                        'n_estimators': [200,
                                                                                         233,
                                                                                         266,
                                                                                         300,
                                                                                         333,
                                                                                         366,
                                                                                         400,
                                                                                         433,
                                                                                         466,
                                                                                         500]}},
               cv=4, early_stop=12, generations=5, offspring_size=12,
               population_size=24, scoring='accuracy', verbosity=2)

In [15]:
rfc_ga_25 = RandomForestClassifier(criterion='gini', max_depth=10, max_features='auto', min_samples_leaf=2, min_samples_split=5, n_estimators=433)
rfc_ga_25.fit(features_df_new_ga,y_res)
rfc_ga_25.score(features_df_new_ga,y_res)

0.7769281914893617

In [16]:
#K-BEST FEATURE Selection 25 =================================================================================
#====================================================================================================
#Feature Selection
fs = SelectKBest(f_classif, k=25)
#fs = SelectKBest(chi2, k=10)
fs.fit(X_res, y_res)
feature_index = fs.get_support(True)
print ("feature index =", feature_index)

#f_value, p_value = f_classif(X_train, y_train)
#print (feature_index.tolist(), f_value, p_value) 

feature index = [  2  25  28  30  32  33  34  39  64  89  91  94  97  98  99 100 105 139
 148 159 172 183 194 281 284]


In [17]:
#Feature selection-Continued
fs = SelectKBest(f_classif, k=25)
new_X_train_25 = fs.fit_transform(X_res,y_res)
print(new_X_train_25.shape)

new_X_train_SelectedFeatures_25= pd.DataFrame(new_X_train_25)
print(new_X_train_SelectedFeatures_25)



(6016, 25)
        0    1    2    3    4    5    6    7    8    9   ...   15   16   17  \
0     38.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0  0.0  3.0   
1     40.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0  0.0  3.0   
2     23.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3     28.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  5.0  ...  3.0  3.0  9.0   
4     23.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0  0.0  3.0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
6011  30.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
6012  27.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  2.0  ...  0.0  0.0  3.0   
6013  26.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
6014  75.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0  0.0  3.0   
6015  61.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

        18   19      20         21    22

In [18]:
#Apply feature selection to test data also

new_X_test_25 = fs.transform(Xtest)
print(new_X_test_25.shape)

new_X_test_SelectedFeatures_25= pd.DataFrame(new_X_test_25)
print(new_X_test_SelectedFeatures_25)



(75818, 25)
         0    1    2    3    4    5    6    7    8    9   ...   15   16   17  \
0      32.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0  0.0  3.0   
1      35.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0  0.0  3.0   
2      23.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  2.0  ...  0.0  0.0  3.0   
3      24.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4      23.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0  0.0  3.0   
...     ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
75813  23.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
75814  26.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0  0.0  3.0   
75815  24.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0  0.0  3.0   
75816  40.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0  0.0  3.0   
75817  23.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  ...  0.0  0.0  3.0   

        18   19   20    21 

In [19]:
#K-BEST FEATURE Selection 50 =================================================================================
#====================================================================================================
#Feature Selection
fs = SelectKBest(f_classif, k=50)
#fs = SelectKBest(chi2, k=10)
fs.fit(X_res, y_res)
feature_index = fs.get_support(True)
print ("feature index =", feature_index)

#f_value, p_value = f_classif(X_train, y_train)
#print (feature_index.tolist(), f_value, p_value) 

feature index = [  2  13  14  17  24  25  28  30  31  32  33  34  39  49  50  64  77  80
  89  91  94  97  98  99 100 105 114 115 139 148 155 159 168 169 172 177
 183 191 194 264 281 283 284 329 332 339 340 341 343 344]


In [20]:
#Feature selection-Continued
fs = SelectKBest(f_classif, k=50)
new_X_train_50 = fs.fit_transform(X_res,y_res)
print(new_X_train_50.shape)

new_X_train_SelectedFeatures_50= pd.DataFrame(new_X_train_50)
print(new_X_train_SelectedFeatures_50)



(6016, 50)
        0      1      2      3    4    5    6    7    8    9   ...   40   41  \
0     38.0    0.0    0.0    0.0  1.0  1.0  0.0  1.0  0.0  0.0  ...  3.0  0.0   
1     40.0    0.0    0.0    0.0  1.0  1.0  0.0  0.0  0.0  0.0  ...  2.0  0.0   
2     23.0    0.0    0.0    0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
3     28.0  210.0  210.0  210.0  1.0  1.0  0.0  1.0  1.0  1.0  ...  2.0  2.0   
4     23.0    0.0    0.0    0.0  1.0  1.0  0.0  0.0  0.0  0.0  ...  3.0  0.0   
...    ...    ...    ...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
6011  30.0    0.0    0.0    0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
6012  27.0  540.0  540.0  540.0  1.0  1.0  0.0  0.0  0.0  0.0  ...  3.0  0.0   
6013  26.0    0.0    0.0    0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
6014  75.0    0.0    0.0    0.0  1.0  1.0  0.0  0.0  0.0  0.0  ...  3.0  0.0   
6015  61.0    0.0    0.0    0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0   

       42       43       44 

In [21]:
#Apply feature selection to test data also

new_X_test_50 = fs.transform(Xtest)
print(new_X_test_50.shape)

new_X_test_SelectedFeatures_50= pd.DataFrame(new_X_test_50)
print(new_X_test_SelectedFeatures_50)



(75818, 50)
         0     1     2     3    4    5    6    7    8    9   ...   40   41  \
0      32.0   0.0   0.0   0.0  1.0  1.0  0.0  0.0  0.0  0.0  ...  3.0  0.0   
1      35.0   0.0   0.0   0.0  1.0  1.0  0.0  0.0  0.0  0.0  ...  3.0  0.0   
2      23.0  60.0  60.0  60.0  1.0  1.0  0.0  0.0  0.0  0.0  ...  3.0  0.0   
3      24.0   0.0   0.0   0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
4      23.0   0.0   0.0   0.0  1.0  1.0  0.0  0.0  0.0  0.0  ...  3.0  0.0   
...     ...   ...   ...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
75813  23.0   0.0   0.0   0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
75814  26.0   0.0   0.0   0.0  1.0  1.0  0.0  0.0  0.0  0.0  ...  3.0  0.0   
75815  24.0   0.0   0.0   0.0  1.0  1.0  0.0  0.0  0.0  0.0  ...  3.0  0.0   
75816  40.0   0.0   0.0   0.0  1.0  1.0  0.0  0.0  0.0  0.0  ...  3.0  0.0   
75817  23.0   0.0   0.0   0.0  1.0  1.0  0.0  0.0  0.0  0.0  ...  3.0  0.0   

        42    43     44   45   46   47   48   49  


In [22]:
#genetic algorithm (GA) for hyperparameter tuning for Random forest classifier with 50 features
from tpot import TPOTClassifier


tpot_classifier_RFC_25 = TPOTClassifier(generations= 5, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestClassifier': param}, 
                                 cv = 4, scoring = 'accuracy')
tpot_classifier.fit(new_X_train_25,y_res)

Optimization Progress:   0%|          | 0/84 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7448470744680851

Generation 2 - Current best internal CV score: 0.7448470744680851

Generation 3 - Current best internal CV score: 0.7448470744680851

Generation 4 - Current best internal CV score: 0.7448470744680851

Generation 5 - Current best internal CV score: 0.745844414893617

Best pipeline: RandomForestClassifier(CombineDFs(input_matrix, input_matrix), criterion=gini, max_depth=560, max_features=auto, min_samples_leaf=6, min_samples_split=10, n_estimators=200)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'criterion': ['entropy',
                                                                                      'gini'],
                                                                        'max_depth': [10,
                                                                                      120,
                                                                                      230,
                                                                                      340,
                                                                                      450,
                                                                                      560,
                                                                                      670,
                                                                                      780,
                                                                                      890,
                                                                                      1000],
                                                                        'max_features': ['auto',
                                                                                         'sqrt',
                                                                                         'log2'],
                                                                        'min_samples_leaf': [1,
                                                                                             2,
                                                                                             4,
                                                                                             6,
                                                                                             8],
                                                                        'min_samples_split': [2,
                                                                                              5,
                                                                                              10,
                                                                                              14],
                                                                        'n_estimators': [200,
                                                                                         233,
                                                                                         266,
                                                                                         300,
                                                                                         333,
                                                                                         366,
                                                                                         400,
                                                                                         433,
                                                                                         466,
                                                                                         500]}},
               cv=4, early_stop=12, generations=5, offspring_size=12,
               population_size=24, scoring='accuracy', verbosity=2)

In [23]:
rfc_25 = RandomForestClassifier(criterion='entropy', max_depth=120, max_features='sqrt', min_samples_leaf=8, min_samples_split=2, n_estimators=400)
rfc_25.fit(new_X_train_25,y_res)
print('Accuracy of Random Forest with 25 Features')
rfc_25.score(new_X_train_25,y_res)

Accuracy of Random Forest with 25 Features


0.7636303191489362

In [24]:
predictions = rfc_25.predict(new_X_test_25)
submission = test_data[['ID']]
submission['TARGET'] = predictions
submission.to_csv('results_RFC_25', index= None)

In [25]:
#genetic algorithm (GA) for hyperparameter tuning for Random forest classifier with 50 features
from tpot import TPOTClassifier


tpot_classifier_RFC_50 = TPOTClassifier(generations= 5, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestClassifier': param}, 
                                 cv = 4, scoring = 'accuracy')
tpot_classifier.fit(new_X_train_50,y_res)

Optimization Progress:   0%|          | 0/84 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7519946808510638

Generation 2 - Current best internal CV score: 0.7523271276595744

Generation 3 - Current best internal CV score: 0.7523271276595744

Generation 4 - Current best internal CV score: 0.7543218085106382

Generation 5 - Current best internal CV score: 0.7543218085106382

Best pipeline: RandomForestClassifier(CombineDFs(input_matrix, input_matrix), criterion=gini, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=366)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'criterion': ['entropy',
                                                                                      'gini'],
                                                                        'max_depth': [10,
                                                                                      120,
                                                                                      230,
                                                                                      340,
                                                                                      450,
                                                                                      560,
                                                                                      670,
                                                                                      780,
                                                                                      890,
                                                                                      1000],
                                                                        'max_features': ['auto',
                                                                                         'sqrt',
                                                                                         'log2'],
                                                                        'min_samples_leaf': [1,
                                                                                             2,
                                                                                             4,
                                                                                             6,
                                                                                             8],
                                                                        'min_samples_split': [2,
                                                                                              5,
                                                                                              10,
                                                                                              14],
                                                                        'n_estimators': [200,
                                                                                         233,
                                                                                         266,
                                                                                         300,
                                                                                         333,
                                                                                         366,
                                                                                         400,
                                                                                         433,
                                                                                         466,
                                                                                         500]}},
               cv=4, early_stop=12, generations=5, offspring_size=12,
               population_size=24, scoring='accuracy', verbosity=2)

In [26]:
rfc_50 = RandomForestClassifier(criterion='gini', max_depth=230, max_features='sqrt', min_samples_leaf=8, min_samples_split=10, n_estimators=400)
rfc_50.fit(new_X_train_50,y_res)
print('Accuracy of Random Forest with 50 Features')
rfc_50.score(new_X_train_50,y_res)

Accuracy of Random Forest with 50 Features


0.7787566489361702

In [27]:
predictions = rfc_50.predict(new_X_test_50)
submission = test_data[['ID']]
submission['TARGET'] = predictions
submission.to_csv('results_RFC_50', index= None)

In [28]:
#RANDOM SEARCH--------------------------------------------

import time
start_time = time.time()

print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_leaf' : range(10,300,10),
            'max_depth': range(5,30,2),
            'criterion':['gini','entropy']}


print("--- %s seconds ---" % (time.time() - start_time))

RandomizedSearchCV-Decision tree
--- 0.0009961128234863281 seconds ---


In [29]:
from tpot import TPOTClassifier


tpot_classifier_DT_25 = TPOTClassifier(generations= 2, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.tree.DecisionTreeClassifier': parameters}, 
                                 cv = 4, scoring = 'accuracy')
tpot_classifier.fit(new_X_train_25,y_res)

Optimization Progress:   0%|          | 0/84 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7468417553191489

Generation 2 - Current best internal CV score: 0.7468417553191489

Generation 3 - Current best internal CV score: 0.7468417553191489

Generation 4 - Current best internal CV score: 0.7468417553191489

Generation 5 - Current best internal CV score: 0.7468417553191489

Best pipeline: RandomForestClassifier(CombineDFs(input_matrix, input_matrix), criterion=entropy, max_depth=890, max_features=sqrt, min_samples_leaf=6, min_samples_split=14, n_estimators=433)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'criterion': ['entropy',
                                                                                      'gini'],
                                                                        'max_depth': [10,
                                                                                      120,
                                                                                      230,
                                                                                      340,
                                                                                      450,
                                                                                      560,
                                                                                      670,
                                                                                      780,
                                                                                      890,
                                                                                      1000],
                                                                        'max_features': ['auto',
                                                                                         'sqrt',
                                                                                         'log2'],
                                                                        'min_samples_leaf': [1,
                                                                                             2,
                                                                                             4,
                                                                                             6,
                                                                                             8],
                                                                        'min_samples_split': [2,
                                                                                              5,
                                                                                              10,
                                                                                              14],
                                                                        'n_estimators': [200,
                                                                                         233,
                                                                                         266,
                                                                                         300,
                                                                                         333,
                                                                                         366,
                                                                                         400,
                                                                                         433,
                                                                                         466,
                                                                                         500]}},
               cv=4, early_stop=12, generations=5, offspring_size=12,
               population_size=24, scoring='accuracy', verbosity=2)

In [30]:
dt_25= DecisionTreeClassifier(criterion='entropy', max_depth=670, max_features='auto', min_samples_leaf=8, min_samples_split=5)
dt_25.fit(new_X_train_25,y_res)
dt_25.score(new_X_train_25,y_res)

0.7172539893617021

In [31]:
predictions = dt_25.predict(new_X_test_25)
submission = test_data[['ID']]
submission['TARGET'] = predictions
submission.to_csv('results_DT_25', index= None)

In [32]:
from tpot import TPOTClassifier


tpot_classifier_DT_50 = TPOTClassifier(generations= 2, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.tree.DecisionTreeClassifier': parameters}, 
                                 cv = 4, scoring = 'accuracy')
tpot_classifier_DT_50.fit(new_X_train_50,y_res)

Optimization Progress:   0%|          | 0/48 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7473404255319149

Generation 2 - Current best internal CV score: 0.7473404255319149

Best pipeline: DecisionTreeClassifier(input_matrix, criterion=entropy, max_depth=27, min_samples_leaf=150)


TPOTClassifier(config_dict={'sklearn.tree.DecisionTreeClassifier': {'criterion': ['gini',
                                                                                  'entropy'],
                                                                    'max_depth': range(5, 30, 2),
                                                                    'min_samples_leaf': range(10, 300, 10)}},
               cv=4, early_stop=12, generations=2, offspring_size=12,
               population_size=24, scoring='accuracy', verbosity=2)

In [33]:
dt_50= DecisionTreeClassifier(criterion='entropy', max_depth=670, max_features='auto', min_samples_leaf=8, min_samples_split=5)
dt_50.fit(new_X_train_50,y_res)
dt_50.score(new_X_train_50,y_res)

0.7603058510638298

In [34]:
predictions = dt_50.predict(new_X_test_50)
submission = test_data[['ID']]
submission['TARGET'] = predictions
submission.to_csv('results_DT_50', index= None)